## Affordability Index — What Does It Mean?

The **Affordability Index** tracks how easily Albertans can afford to live each month. It combines wage growth, inflation, rent changes, and unemployment trends into a single score.

- **Above 0:** Affordability has improved
- **Below 0:** Affordability has worsened

### Key Insights

- **2020:** A sharp drop highlights the economic shock of COVID-19.
- **2021:** Affordability recovers.
- **2023–2024:** Rising rents and inflation pressures cause renewed instability.

In [0]:
%sql
SELECT
  date,
  affordability_index
FROM mycatalog.gold.affordability_gold
ORDER BY date;


Databricks visualization. Run in Databricks to view.

## Wages, Inflation, and Rent — What’s Driving Affordability?

This chart tracks how Alberta’s key cost-of-living factors have shifted over time:

- **Wages:** Grew steadily, but gains were modest.
- **Inflation:** Spiked sharply in 2021–2022, eroding purchasing power.
- **Rent:** Accelerated after 2022, adding pressure on household budgets.

These trends together help explain the major movements in the Affordability Index.

In [0]:
%sql
SELECT 
    date,
    wage_growth,
    inflation_rate,
    rent_change_percentage AS rent_change
FROM mycatalog.gold.affordability_gold
ORDER BY date;


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT 
    wage_growth,
    inflation_rate,
    rent_change_percentage AS rent_change,
    unemployment_rate_change,
    affordability_index
FROM mycatalog.gold.affordability_gold
WHERE wage_growth IS NOT NULL
ORDER BY date;


In [0]:
pdf = _sqldf.toPandas()


##Correlation Cluster Heatmap

## Economic Relationships Driving Affordability in Alberta

This heatmap visualizes how Alberta’s key economic indicators—wage growth, inflation, rent changes, and unemployment—interact to shape the Affordability Index. 

- **Positive values** (red): Indicators move together, amplifying affordability shifts.
- **Negative values** (blue): Indicators move in opposite directions, offsetting each other’s impact.

**Key takeaways:**
- Strong negative correlation between inflation and affordability: Higher inflation sharply reduces affordability.
- Wage growth and affordability are positively linked, but not enough to fully offset inflation and rent pressures.
- Rent changes and unemployment also play important roles, with complex interactions.

Explore the chart to see which factors most strongly drive affordability trends in Alberta.

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Load gold table into pandas
df = spark.table("mycatalog.gold.affordability_gold").toPandas()

# Select only numeric columns for correlation
cols = [
    "wage_growth",
    "inflation_rate",
    "rent_change_percentage",
    "unemployment_rate_change",
    "affordability_index"
]

corr = df[cols].corr()

# Better styling
plt.figure(figsize=(12, 8))
sns.heatmap(
    corr,
    annot=True,
    cmap="coolwarm",
    vmin=-1,
    vmax=1,
    fmt=".2f",
    linewidths=0.7,
    square=True,
    cbar_kws={"shrink": 0.75}
)

plt.title(
    "Correlation Heatmap — Economic Drivers Behind Alberta's Affordability Index",
    fontsize=16,
    fontweight='bold',
    pad=20
)
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


In [0]:
%sql
CREATE OR REPLACE TEMP VIEW affordability_regimes AS
SELECT
  date,
  inflation_rate,
  affordability_index,
  CASE
    WHEN affordability_index >= 5 THEN 'High affordability'
    WHEN affordability_index <= -5 THEN 'Low affordability'
    ELSE 'Medium affordability'
  END AS affordability_regime
FROM mycatalog.gold.affordability_gold
WHERE affordability_index IS NOT NULL
  AND inflation_rate IS NOT NULL;


In [0]:
%sql
SELECT
  inflation_rate,
  affordability_index,
  affordability_regime
FROM affordability_regimes;



Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT * FROM mycatalog.gold.affordability_regimes LIMIT 10;
